In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
v_file_date

Out[133]: '2021-04-18'

In [0]:
%run ../includes/configuration

In [0]:
drivers_df = spark.read.parquet(f'{processed_folder_path}drivers').\
withColumnRenamed('number','driver_number').\
withColumnRenamed('name','driver_name').\
withColumnRenamed('nationality','driver_nationality').\
withColumnRenamed('driver-id','driver_id')

In [0]:
race_df = spark.read.parquet(f'{processed_folder_path}race').\
withColumnRenamed('name','race_name').\
withColumnRenamed('race_timestamp','race_date')


In [0]:
constructors_df = spark.read.parquet(f'{processed_folder_path}constructors').\
withColumnRenamed('name','team').\
withColumnRenamed('nationality','constructor_nationality')

In [0]:
circuits_df = spark.read.parquet(f'{processed_folder_path}circuits').\
withColumnRenamed('location','circuit_location')

In [0]:
results_df = spark.read.parquet(f'{processed_folder_path}results').\
filter(f"file_date == '{v_file_date}'").\
withColumnRenamed('time','race_time').\
withColumnRenamed('race_id','result_race_id').\
withColumnRenamed('file_date','result_file_date')

In [0]:
race_circuit_df = race_df.join(circuits_df, race_df.circuit_id == circuits_df.circuit_id, 'inner').\
select(race_df.race_id, race_df.race_year, race_df.race_name, race_df.race_date, circuits_df.circuit_location)

In [0]:
race_results_df = results_df.join(drivers_df, drivers_df.driver_id == results_df.driver_id)\
                  .join(race_circuit_df, race_circuit_df.race_id == (results_df.result_race_id))\
                .join(constructors_df, constructors_df.constructor_id == results_df.constructor_id)

In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
final_df = race_results_df.select('race_id','race_year', 'race_name', 'race_date', 'circuit_location', 'driver_name','driver_number', 'driver_nationality', 'team', 'grid', 'fastest_lap', 'race_time', 'points', 'position','result_file_date').\
withColumn('created_time', current_timestamp()).\
withColumnRenamed('result_file_date', 'file_date')

In [0]:
#display(final_df)

In [0]:
#display(final_df.filter("race_year = 2020 and race_name = 'Abu Dhabi Grand Prix'").orderBy(final_df.points.desc()))


In [0]:
%run "../includes/common_functions"

In [0]:
overwrite_partition(final_df, 'race_id', 'f1_presentation', 'race_results')

In [0]:
# %sql
# SELECT race_id, count(1)
# FROM f1_presentation.race_results
# GROUP BY race_id
# ORDER BY race_id DESC